### Notebook to read and clean the GDFL Data

Purpose: Read the GDFL FNO Data CSV file and process the data to save different types of files as below: 
1. Nifty Options - (Nifty_Date_Options)
2. Nifty Futures - (Nifty_Date_Futures)
3. BankNifty Options - (BN_Date_Options)
4. BankNifty Futures - (BN_Date_Futures)
5. Equity Options - (EQ_Date_Options)
6. Equity Futures - (EQ_Date_Futures)

### Pseudo Code 

1. Read raw CSV File and extract Symbol from the Ticker column
2. Assign Index Type as EQFNO or IXFNPO based on the Symbol 
3. Assign Type = Fut or CE or PE based on Ticker column
4. If Index_Type==Nifty 
        a. If Type==Fut
            i. Extract Contract Types  - -I, -II,-III
            ii. Write to file - Nifty_Date_Futures
        b. Elif Type!=Fut
            i. If Date<Nifty Start Date
                  Extract Monthly Expiries and Strikes 
            ii. elif Date>Nifty Start Date
                  Extract Weekly Expiries and Strikes
            iii. Write to file - Nifty_Date_Options
5. Elif Index_Type == BankNifty
        a. If Type==Fut
            i. Extract Contract Types  - -I, -II,-III
            ii. Write to file - BN_Date_Futures
        b. Elif Type!=Fut
            i. If Date<BankNifty Start Date
                  Extract Monthly Expiries and Strikes 
            ii. elif Date>BankNifty Start Date
                  Extract Weekly Expiries and Strikes
            iii. Write to file - BN_Date_Options
6. Elif Index_Type==EQFNO
        a. if Type==Fut
            i. Exract Contract Types -> -I, -II, -III
            ii. Write to file - EQ_Date_Futures
        b. elif Type!=Fut
            i. if Date<Nifty Start Date
                - Extract Monthly Expiries and Strikes
                - Handle exceptions for Decimel Strikes 
            ii. else
                 - Extract Weekly Expiries and Strikes
                 - Handle exceptions for Decimel strikes
            iii. Write to file - EQ_Date_Options
                             

In [1]:
import csv
import pandas as pd
import datetime as dt
import numpy as np
import os
import math
import time
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Nifty Weekly Start Dates
nifty_weekly_start=dt.datetime(year=2017,month=2,day=11,hour=0,minute=0,second=0)

In [3]:
# BankNifty Weekly Start Dates
bn_weekly_start=dt.datetime(year=2016,month=5,day=27)

In [4]:
## Enter the Directory where the final pkl files are to be stored
save_directory=r'C:\Users\ankit\Documents\Jupyter Notebooks\Systematic Trading - BackTests\Data Pickles\GDFL Clean Data'

In [5]:
## Enter the source data Directory from where to read data
src_directory=r'C:\Users\ankit\Dropbox\Ventures\Trading\NSE Historical Data\Options Datamart Code'

In [6]:
## A function to read String format of Date and Time columns and create a unique DateTime type
## Inputs - DataFrame of Raw Data
## Outputs - Retruns Column with DateTime type

def extract_datetime(df):
    df['DateTime']=df['Date']+' '+df['Time']
    df['DateTime']=df.DateTime.apply(lambda x:dt.datetime.strptime(x,'%d/%m/%Y %H:%M:%S'))
    print('Cleaning Data for:'+df['Date'][0])
    return

In [7]:
## A function to read String format of Ticker columns and create two columns 
## - Column Type - has values as Fut, CE or PE
## - Column Index - has values IXFNO or EQFNO 
## Inputs - DataFrame of Data
## Outputs - Retuns 2 columns 
## - data['Type'] and data['Index']

def extract_indextype(df):
    df['Ticker']=df['Ticker'].str.upper()
    
    ## Split df to pick up CE and PE as Type from the Ticker Name
    df['Type']=df['Ticker'].apply(lambda x:x.split('.NFO')[0][-2:])
    
    ## Replace -I, II,III with Fut
    df['Type']=np.where((df['Type']!='CE')&(df['Type']!='PE'),'Fut',df['Type'])
    
    ## All Index Tickers have NIFTY in it - NIFTY, BANKNIFTY, NIFTYIT, NIFTYINFRA, MINIFTY
    df['Index']=df['Ticker'].apply(lambda x:'IXFNO' if 'NIFTY' in x else 'EQFNO')
    
    return

In [8]:
def extract_symbol(df):
    # Read the Ticker and identify the Symbol Names from it
    # This uses REGEX within str.extract() method to identify the pattern
    # For Type_name==OPT, fomat is as above noted
    # \w matches AlphaNumeric characters, followed by '+' matches multiple length
    # [0-9] matches a digit or can also use \d 
    # To Match futures - ACC-I; "-" is used as an escaping character
    df['Symbol']=np.where(df['Type']!='Fut',\
                        df['Ticker'].str.extract(r'([A-Za-z0-9-&]+[0-9][0-9][A-Z][A-Z][A-Z])',expand=True)[0].astype(str).\
                        apply(lambda x:x[:-5]),
                        df['Ticker'].str.extract(r'([A-Za-z0-9-&]+\-)',expand=True)[0].astype(str).apply(lambda x:x[:-1]))
    return

In [9]:
## Extract Future Types (I Month, II Second Month, III Month)
def extract_contract(df):
    df['Contract']=df['Ticker'].apply(lambda x:x.split('-')[-1].split('.')[0])
    return

In [10]:
def extract_weekly(df):
    ## Example: NIFTY24JUN2110600CE.NFO
    df['Expiry']=pd.to_datetime(
                    df['Ticker'].str.extract(r'([0-9][0-9][A-Z][A-Z][A-Z]\d+[A-Z][A-Z])')[0].\
                    astype(str).apply(lambda x:x[:7]),\
                    format='%d%b%y')
    df['Strike']=df['Ticker'].str.extract(r'([0-9][0-9][A-Z][A-Z][A-Z]\d+[A-Z][A-Z])')[0].astype(str).\
                        apply(lambda x:x[7:-2])
    return

In [11]:
def extract_monthly(df):
    ## Example: NIFTY16JUN10600CE.NFO
    df['Expiry']=pd.to_datetime(
                    df['Ticker'].str.extract(r'(\d\d[A-Z][A-Z][A-Z]\d+[A-Z][A-Z])')[0].astype(str).apply(lambda x:x[:5]),\
                    format='%y%b')
    df['Strike']=df['Ticker'].str.extract(r'(\d\d[A-Z][A-Z][A-Z]\d+[A-Z][A-Z])')[0].astype(str).apply(lambda x:x[5:-2])
    return

In [12]:
## Function to create a slice of main data for Nifty futures and apply appropriate features 
## Save the cleaned slice of data as a separate pickle file
def clean_nifty_future(df):
    nifty_fut=df[(df['Symbol']=='NIFTY') & (df['Type']=='Fut')]
    nifty_fut.reset_index(inplace=True,drop=True)
    extract_contract(nifty_fut)
    date=nifty_fut['DateTime'][0].strftime(format='%Y-%m-%d')
    filename='nifty_%s_fut.pkl'%(date)
    nifty_fut.to_pickle(save_directory+'\\'+filename)
    #print('Saved Nifty Future for:'+date)
    return

In [13]:
## Function to create a slice of main data for Nifty Options and create appropriate expiry and strike columns
## Save the data in appropriate files
def clean_nifty_opt(df):
    nifty_opt=data[(data['Symbol']=='NIFTY') & (data['Type']!='Fut')]
    nifty_opt.reset_index(inplace=True,drop=True)

    if (nifty_opt['DateTime'][0]>=nifty_weekly_start):
        extract_weekly(nifty_opt)
    else:
        extract_monthly(nifty_opt)
    date=nifty_opt['DateTime'][0].strftime(format='%Y-%m-%d')
    filename='nifty_%s_opt.pkl'%(date)
    nifty_opt.to_pickle(save_directory+'\\'+filename)
    #print('Saved Nifty Options for:'+date)
    return

In [14]:
## Function to create a slice of main data for BanNifty futures and apply appropriate features 
## Save the cleaned slice of data as a separate pickle file
def clean_bn_future(df):
    bn_fut=df[(df['Symbol']=='BANKNIFTY') & (df['Type']=='Fut')]
    bn_fut.reset_index(inplace=True,drop=True)
    extract_contract(bn_fut)
    date=bn_fut['DateTime'][0].strftime(format='%Y-%m-%d')
    filename='bn_%s_fut.pkl'%(date)
    bn_fut.to_pickle(save_directory+'\\'+filename)
    #print('Saved BankNifty Future for:'+date)
    return

In [15]:
## Function to create a slice of main data for Nifty Options and create appropriate expiry and strike columns
## Save the data in appropriate files
def clean_bn_opt(df):
    bn_opt=data[(data['Symbol']=='BANKNIFTY') & (data['Type']!='Fut')]
    bn_opt.reset_index(inplace=True,drop=True)

    if (bn_opt['DateTime'][0]>=bn_weekly_start):
        extract_weekly(bn_opt)
    else:
        extract_monthly(bn_opt)
    date=bn_opt['DateTime'][0].strftime(format='%Y-%m-%d')
    filename='bn_%s_opt.pkl'%(date)
    bn_opt.to_pickle(save_directory+'\\'+filename)
    #print('Saved BankNifty Options for:'+date)
    return

In [16]:
## Function to create a slice of main data that does not match Nifty or BanNifty futures and apply appropriate features 
## Save the cleaned slice of data as a separate pickle file
def clean_eq_future(df):
    eq_fut=data[(data['Symbol']!='BANKNIFTY')&(data['Symbol']!='NIFTY')&(data['Type']=='Fut')]
    eq_fut.reset_index(inplace=True,drop=True)
    extract_contract(eq_fut)
    date=eq_fut['DateTime'][0].strftime(format='%Y-%m-%d')
    filename='eq_%s_fut.pkl'%(date)
    eq_fut.to_pickle(save_directory+'\\'+filename)
    #print('Saved EQ Future for:'+date)
    return

In [17]:
## Stocks may have decimal strikes and those need to be addressed
## The function below intakes a dataframe and date (weekly start date)
## it extracts decimal strikes and then convert expiries to datetime
## previous expiries were converted for non-decimal strikes and thus needs special treatment in code
def extract_decimal(df,date):
    if(date>=nifty_weekly_start):
        df['Expiry']=np.where(df['Expiry'].isnull(),\
                                    df['Ticker'].str.extract(r'([0-9][0-9][A-Z][A-Z][A-Z]\d+\.\d+[A-Z][A-Z])')[0].\
                                        astype(str).apply(lambda x:x[:7]),\
                                df.Expiry)
        
        ## The above code results in a mixed column of expiry data types
        # First type is date of format : 23FEB19
        date1=pd.to_datetime(df['Expiry'],format='%d%b%y',errors='coerce')
        # Second type is previously converted dates with timestamps in nanoseconds from epoch
        date2=pd.to_datetime(df['Expiry'],unit='ns',errors='coerce')
        #after converting both types to datetime, use fillna to combine them together
        df['Expiry']=date1.fillna(date2)
        
        df['Strike']=np.where(df['Strike']=='',\
                                  df['Ticker'].str.extract(r'([0-9][0-9][A-Z][A-Z][A-Z]\d+\.\d+[A-Z][A-Z])')[0].\
                                  astype(str).apply(lambda x:x[7:-2]),\
                              df.Strike)
    else:
        df['Expiry']=np.where(df['Expiry'].isnull(),\
                                  df['Ticker'].str.extract(r'([0-9][0-9][A-Z][A-Z][A-Z]\d+\.\d+[A-Z][A-Z])')[0].\
                                  astype(str).apply(lambda x:x[:5]),\
                              df.Expiry)
        ## The above code results in a mixed column of expiry data types
        # First type is date of format : 16FEB
        date1=pd.to_datetime(df['Expiry'],format='%y%b',errors='coerce')
        # Second type is previously converted dates with timestamps in nanoseconds from epoch
        date2=pd.to_datetime(df['Expiry'],unit='ns',errors='coerce')
        #after converting both types to datetime, use fillna to combine them together
        df['Expiry']=date1.fillna(date2)
        
        df['Strike']=np.where(df['Strike']=='',\
                                  df['Ticker'].str.extract(r'([0-9][0-9][A-Z][A-Z][A-Z]\d+\.\d+[A-Z][A-Z])')[0].\
                                  astype(str).apply(lambda x:x[5:-2]),\
                              df.Strike)

In [18]:
## Function to create a slice of main data that does not match Nifty or BanNifty options and apply appropriate features 
## Save the cleaned slice of data as a separate pickle file
def clean_eq_opt(df):
    eq_opt=data[(data['Symbol']!='BANKNIFTY')&(data['Symbol']!='NIFTY')&(data['Type']!='Fut')]
    eq_opt.reset_index(inplace=True,drop=True)

    if(eq_opt['DateTime'][0]>=nifty_weekly_start):
        extract_weekly(eq_opt)
    else:
        extract_monthly(eq_opt)
    ## Stock options may have decimal strikes and needs to be cleaned up
    extract_decimal(eq_opt,eq_opt['DateTime'][0])
    date=eq_opt['DateTime'][0].strftime(format='%Y-%m-%d')
    filename='eq_%s_opt.pkl'%(date)
    eq_opt.to_pickle(save_directory+'\\'+filename)
    #print('Saved EQ Option for:'+date)
    return

In [19]:
def clean_data(df):
    extract_datetime(df)
    extract_indextype(df)
    extract_symbol(df)
    clean_nifty_future(df)
    clean_nifty_opt(df)
    clean_bn_future(df)
    clean_bn_opt(df)
    clean_eq_future(df)
    clean_eq_opt(df)
    return

In [20]:
## This functions taken two inputs
def new_date_check(dates_list,search_date):    
    flag=True
    for d in dates_list:
        if d==search_date:
            flag=False
            break
    return flag

In [21]:
# Get the current working directory and save it in a variable. This will be used to switch back after we have read data
current_directory=os.getcwd()

#Change the working directory to the path provided above. 
os.chdir(src_directory)

# Read all pkl files stores in the directory where clean data is stored 
clean_data_files=os.listdir(save_directory)

# Extract the list of unique dates for which we already have the data
dates_list=[dt.datetime.strptime(str(x.split('_')[-2]),'%Y-%m-%d') for x in clean_data_files]
dates_list=np.unique(dates_list)

#Run a Loop that lists total no of files within the directory path. 
#It is important that there are only readable data in the same folder


for file1 in os.listdir(src_directory):
    
    #read the csv into an empty new variable - df
    data=pd.read_csv(file1,sep=',')
    
    search_date=dt.datetime.strptime(data['Date'][0],'%d/%m/%Y')
    
    if(new_date_check(dates_list,search_date)):
        print('Cleaning Data For: '+str(search_date))
        clean_data(data)
    else:
        print('Clean Data already exists for: '+str(search_date)+'.....Reading next file')
        continue
        
#Revert the working directory to the original directory from wher we started
os.chdir(current_directory)

#os.listdir(src_directory)

Cleaning Data For: 2019-02-01 00:00:00
Cleaning Data for:1/2/2019
Cleaning Data For: 2021-03-12 00:00:00
Cleaning Data for:12/3/2021
Cleaning Data For: 2011-04-01 00:00:00
Cleaning Data for:1/4/2011
Cleaning Data For: 2016-01-15 00:00:00
Cleaning Data for:15/01/2016
Cleaning Data For: 2016-06-16 00:00:00
Cleaning Data for:16/06/2016
